# Data Source
- Select Dataset concluded years
- Feature drop 
    - Unique keys, redundant features
    - ratio of missing value > 95%
    - Select Feature two methods
        - select features 15~18 how to pre-train xgboost method
        - bivariate, correlation analysis
            - borrowers characteristic
            - Loan characteristic
            - credit characteristic
            - credit history 
    - Data augmentation
        - Random undersampling
        - Oversampling SMOTE-NC
        - Generation exclude based AE
    - Machig Learning Modeling
        - LR
        - xbgoost
        - LightGBM
    - Evaluation
        - G-mean score
        - K-S score
        - Confusion matrix
        - ROC - AUC

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import os

import torch

import warnings
warnings.filterwarnings("ignore") 

In [2]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

print(device)

cuda


In [3]:
## Load lending club data 
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/loan_data.csv"
data = pd.read_csv(data_path, low_memory=False)

data = data[data['loan_status'] == 1]

In [5]:
len(data.columns)

142